In [11]:
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws

import pandas as pd
import numpy as np
import math

import datetime as dt
from datetime import date, timedelta,datetime
import time 

from fyers_apiv3.FyersWebsocket import data_ws
import csv

import matplotlib.pyplot as plt
import mplfinance as mpf

import scipy

from  time import sleep
import threading
import os
import pyotp
import requests
import json
import pytz
from urllib.parse import parse_qs,urlparse
import base64

import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [12]:
redirect_uri = "https://trade.fyers.in/api-login/redirect-uri/index.html"
client_id='HTHFWXFS49-100'
secret_key = 'JOQVVTY9UJ'
FY_ID = "YB00632"  
TOTP_KEY = "3Y7PAJSPNHXEADGRDD726MLHJW43AJGH"  
PIN = "1964" 

grant_type = "authorization_code"                  
response_type = "code"                         
state = "sample"                                  

appSession = fyersModel.SessionModel(client_id = client_id, redirect_uri = redirect_uri,response_type=response_type,state=state,secret_key=secret_key,grant_type=grant_type)


generateTokenUrl = appSession.generate_authcode()
generateTokenUrl


def getEncodedString(string):
    string = str(string)
    base64_bytes = base64.b64encode(string.encode("ascii"))
    return base64_bytes.decode("ascii")
  



URL_SEND_LOGIN_OTP="https://api-t2.fyers.in/vagator/v2/send_login_otp_v2"
res = requests.post(url=URL_SEND_LOGIN_OTP, json={"fy_id":getEncodedString(FY_ID),"app_id":"2"}).json()   

if datetime.now().second % 30 > 27 : sleep(5)
URL_VERIFY_OTP="https://api-t2.fyers.in/vagator/v2/verify_otp"
res2 = requests.post(url=URL_VERIFY_OTP, json= {"request_key":res["request_key"],"otp":pyotp.TOTP(TOTP_KEY).now()}).json()  


ses = requests.Session()
URL_VERIFY_OTP2="https://api-t2.fyers.in/vagator/v2/verify_pin_v2"
payload2 = {"request_key": res2["request_key"],"identity_type":"pin","identifier":getEncodedString(PIN)}
res3 = ses.post(url=URL_VERIFY_OTP2, json= payload2).json()  


ses.headers.update({
    'authorization': f"Bearer {res3['data']['access_token']}"
})


TOKENURL="https://api-t1.fyers.in/api/v3/token"
payload3 = {"fyers_id":FY_ID,
           "app_id":client_id[:-4],
           "redirect_uri":redirect_uri,
           "appType":"100","code_challenge":"",
           "state":"None","scope":"","nonce":"","response_type":"code","create_cookie":True}

res3 = ses.post(url=TOKENURL, json= payload3).json()  


url = res3['Url']
parsed = urlparse(url)
auth_code = parse_qs(parsed.query)['auth_code'][0]


grant_type = "authorization_code" 

response_type = "code"  

session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)

session.set_token(auth_code)

response = session.generate_token()

access_token = response['access_token']

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path=os.getcwd())
fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'YB00632',
  'name': 'BABAN MARUTI PAWAR',
  'image': None,
  'display_name': None,
  'pin_change_date': '21-12-2023 21:01:26',
  'email_id': 'babanpawar8975@gmail.com',
  'pwd_change_date': None,
  'PAN': '---------',
  'mobile_number': '8975963982',
  'totp': True,
  'pwd_to_expire': 90}}

In [13]:
def getdata(symbols, res, rfrom, rto):
    cdata = {
            "symbol":sym,
            "resolution":str(res),
            "date_format":"1",
            "range_from":rfrom,
            "range_to": rto,
            "cont_flag":"0"
            } 
    response = fyers.history(data=cdata)
    data = pd.DataFrame.from_dict(response['candles'])
    cols = ['datetime', 'open', 'high','low','close', 'volume']
    data.columns = cols
    data['datetime'] = pd.to_datetime(data['datetime'], unit="s")
    data['datetime'] = data['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    data['datetime'] = data['datetime'].dt.tz_localize(None)
    data = data.set_index('datetime')
    return data

In [14]:
#INITIAL VARIABLE
bpos = 0
spos = 0
flag = 0
currentdate = date.strftime(date.today(), '%Y-%m-%d')
print(currentdate)
isb = False
row = -2
cstrike = ''
pstrike = ''
expiry = 'NSE:NIFTY5024929'
bentry = 0
sentry = 0
bstoploss = 0
sstoploss = 0
btarget = 0
starget = 0
rrr = 1
posflag = False
sym = 'NSE:NIFTY50-INDEX'

2024-10-25


In [ ]:
#ISB STRATEGY FUNCTION
def onmessage(message):
    #symb = message['symbol']
    #ltp = message['ltp']
    #print(message)
    t = time.localtime()
    chrs = time.strftime("%H", t)
    cmin = time.strftime("%M", t)
    csec = time.strftime("%S", t)
    global pos , currentdate, isb, flag, bpos, spos, row, expriy, bentry , sentry, bstoploss, sstoploss, btarget, starget, rrr, cstrike, pstrike, posflag
    if (int(cmin) % 1 == 0  and int(csec) > 1 and flag == 0 and (int(chrs) >= 9 and  int(cmin) >= 17)):
        print("data updated")
        df = getdata('NSE:NIFTY50-INDEX',1, currentdate,currentdate)
        flag = 1
        posflag = False
    if (int(cmin) % 1 != 0 and flag == 1):
        flag = 0
    if (isb == False and df['high'].iloc[row] < df['high'].iloc[row-1]
        and  df['low'].iloc[row] > df['low'].iloc[row-1] ) :
        isbhigh = df['high'].iloc[row-1]
        isblow = df['low'].iloc[row-1]
        isb = True
    # orbhigh = df['high'].iloc[0]
    # orblow = df['low'].iloc[0]
    print(f"{message}")
    if (isb == True and posflag == False and  message['ltp'] > isbhigh and (bpos == 0 or spos == 0)):
        bpos = 1
        print('buy entry')
        ltp = message['ltp']
        sp = int(round(ltp / 50) * 50)
        cstrike = expiry + str(sp) + "CE"
        cdata = {
            "symbol": str(cstrike),
            "qty":25,
            "type":2,
            "side":1,
            "productType":"MARGIN",
            "limitPrice":0,
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            }
        posflag = True
        # response = fyers.place_order(data=cdata)
        bentry = isbhigh
        bstoploss = df['low'].iloc[-1] #------>
        btarget = isbhigh + ((bstoploss)*rrr)
        print(f"entry {cstrike}")
    if (isb == True and posflag == False and  message['ltp'] < isblow and (spos == 0 or bpos== 0)):
        spos = 1
        print('sell entry')
        ltp = message['ltp']
        sp = int(round(ltp / 50) * 50)
        pstrike = expiry + str(sp) + "PE"
        data = {
            "symbol": str(pstrike),
            "qty":25,
            "type":2,
            "side":1,
            "productType":"MARGIN",
            "limitPrice":0,
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            }
        posflag = True
        # response = fyers.place_order(data=data)
        sentry = isblow
        sstoploss =  df['high'].iloc[-1]
        starget = isblow - (( sstoploss - isblow)*rrr)
        print(f"entry {pstrike}")
    if (isb == True and message['ltp'] >= btarget and bpos == 1)
        data = {
            "symbol": str(cstrike),
            "qty":25,
            "type":2,
            "side":-1,
            "productType":"MARGIN",
            "limitPrice":0,
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            }
        print(f"call buy target  {cstrike}")
        # response = fyers.place_order(data=data)
        bpos = 0
        isb = False
    if (isb == True and message['ltp'] <= bstoploss and bpos == 1)
        data = {
            "symbol": str(cstrike),
            "qty":25,
            "type":2,
            "side":-1,
            "productType":"MARGIN",
            "limitPrice":0,
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            }
        print(f"call buy stoploss  {cstrike}")
        # response = fyers.place_order(data=data)
        bpos = 0
        isb = False
    if (isb == True and message['ltp'] <= starget and spos == 1)
        data = {
            "symbol": str(pstrike),
            "qty":25,
            "type":2,
            "side":-1,
            "productType":"MARGIN",
            "limitPrice":0,
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            }
        print(f"put buy target  {pstrike}")
        # response = fyers.place_order(data=data)
        spos = 0
        isb = False
    if (isb == True and message['ltp'] >= sstoploss and spos == 1)
        data = {
            "symbol": str(pstrike),
            "qty":25,
            "type":2,
            "side":-1,
            "productType":"MARGIN",
            "limitPrice":0,
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            }
        print(f"put buy stoploss  {pstrike}")
        # response = fyers.place_order(data=data)
        spos = 0
        isb = False
def onerror(message):
    print("Error:", message)
def onclose(message):
    print("Connection closed:", message)
def onopen():
    data_type = "SymbolUpdate"
    symbols = [sym]
    fyersdata.subscribe(symbols=symbols, data_type=data_type)
    fyersdata.keep_running()

# Replace the sample access token with your actual access token obtained from Fyers
access_token = client_id + ":" + tk
# Create a FyersDataSocket instance with the provided parameters
fyersdata = data_ws.FyersDataSocket(
    access_token=access_token,       # Access token in the format "appid:accesstoken"
    log_path="",                     # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=True,                  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,              # Save response in a log file instead of printing it.
    reconnect=True,                  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,               # Callback function to subscribe to data upon connection.
    on_close=onclose,                # Callback function to handle WebSocket connection close events.
    on_error=onerror,                # Callback function to handle WebSocket errors.
    on_message=onmessage             # Callback function to handle incoming messages from the WebSocket.
)
# Establish a connection to the Fyers WebSocket
fyersdata.connect()